### Competitor Analysis Workflow

In [2]:
import dotenv
dotenv.load_dotenv()

from app.settings import init_aiml, init_openai
init_openai()

from app.agents.stage_6_output_production.podcaster.workflow import PodcastWorkflow
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

# Draw all
draw_all_possible_flows(PodcastWorkflow, filename="podcast_flow_all.html")

ModuleNotFoundError: No module named 'app.agents.stage_6_output_production.podcaster.podcaster'

In [4]:
from textwrap import dedent
from typing import List, Optional
from llama_index.core.workflow import Context, Event, StartEvent, StopEvent, Workflow, step
from app.engine.tools import ToolFactory
from app.workflows.single import FunctionCallingAgent, AgentRunResult
from llama_index.core.chat_engine.types import ChatMessage

class FindCompetitorsEvent(Event):
    input: str

class FindHomepageEvent(Event):
    competitors: List[str]

class FindReviewsEvent(Event):
    competitor_data: dict

class AnalyzeCompetitorsEvent(Event):
    competitor_reviews: dict

def create_agent(name: str, chat_history: List[ChatMessage], tools: List[str] = None, system_prompt = "You are a helpful AI assistant.") -> FunctionCallingAgent:
    configured_tools = ToolFactory.from_env(map_result=True)
    available_tools = []

    if tools:
        for tool in tools:
            if tool in configured_tools:
                available_tools.extend(configured_tools[tool])
    
    return FunctionCallingAgent(
        name=name,
        tools=available_tools,
        description=f"Agent responsible for {name}",
        chat_history=chat_history,
        system_prompt=system_prompt
    )

In [ ]:
class CompetitorAnalysisWorkflow(Workflow):
    def __init__(self, timeout: int = 360, chat_history: Optional[List[ChatMessage]] = None):
        super().__init__(timeout=timeout)
        self.chat_history = chat_history or []

    @step()
    async def start(self, ctx: Context, ev: StartEvent) -> FindCompetitorsEvent:
        ctx.data["task"] = ev.input
        return FindCompetitorsEvent(input=ev.input)

    @step()
    async def find_competitors(
        self, ctx: Context, ev: FindCompetitorsEvent, searcher: FunctionCallingAgent
    ) -> FindHomepageEvent:
        result: AgentRunResult = await self.run_agent(ctx, searcher, 
            "Find top competitors for this product. Only return a list of company names: " + ev.input)
        return FindHomepageEvent(competitors=result.response.message.content)

    @step()
    async def find_homepage(
        self, ctx: Context, ev: FindHomepageEvent, scraper: FunctionCallingAgent
    ) -> FindReviewsEvent:
        competitor_data = {}
        for competitor in ev.competitors:
            result = await self.run_agent(ctx, scraper, 
                f"Find and scrape homepage and pricing for {competitor}")
            competitor_data[competitor] = result.response.message.content
        return FindReviewsEvent(competitor_data=competitor_data)

    @step()
    async def find_reviews(
        self, ctx: Context, ev: FindReviewsEvent, reviewer: FunctionCallingAgent
    ) -> AnalyzeCompetitorsEvent:
        competitor_reviews = {}
        for competitor, data in ev.competitor_data.items():
            result = await self.run_agent(ctx, reviewer,
                f"Find and summarize reviews for {competitor}")
            competitor_reviews[competitor] = {
                "data": data,
                "reviews": result.response.message.content
            }
        return AnalyzeCompetitorsEvent(competitor_reviews=competitor_reviews)

    @step()
    async def analyze(
        self, ctx: Context, ev: AnalyzeCompetitorsEvent, analyzer: FunctionCallingAgent
    ) -> StopEvent:
        result = await self.run_agent(ctx, analyzer,
            f"Analyze these competitors and provide a detailed report with citations: {ev.competitor_reviews}")
        return StopEvent(result=result)

    async def run_agent(self, ctx: Context, agent: FunctionCallingAgent, input: str) -> AgentRunResult:
        handler = agent.run(input=input, streaming=False)
        async for event in handler.stream_events():
            if type(event) is not StopEvent:
                ctx.write_event_to_stream(event)
        return await handler

def create_competitor_analysis(chat_history: List[ChatMessage]):
    workflow = CompetitorAnalysisWorkflow(timeout=360, chat_history=chat_history)
    
    workflow.add_workflows(
        searcher=create_agent("competitor_searcher", 
                              chat_history, 
                              system_prompt=dedent("""
                                                    You are an expert at finding competing products and companies.
                                                        Follow these steps:
                                                        1. Given a user's product description, generate relevant SEO keywords for searching for competitor products
                                                        2. Use the search tool to search across multiple platforms using site-specific searches in your query:
                                                        - site:g2.com
                                                        - site:producthunt.com
                                                        - site:reddit.com
                                                        - site:appsumo.com
                                                        - site:alternativeto.net
                                                        3. Only return a clean list of competitor names, nothing else
                                                        Example output: ["Product A", "Product B", "Product C"]
                                                    """), 
                                            tools=["tavily"]),
        scraper=create_agent("website_scraper", chat_history, ["web_reader"]),
        reviewer=create_agent("review_finder", chat_history, ["tavily", "web_reader"]),
        analyzer=create_agent("competitor_analyzer", chat_history),
    )
    
    return workflow

In [ ]:
system_prompts = {
        "competitor_searcher": """You are an expert at finding competing products and companies.
        Follow these steps:
        1. Given a user's product description, generate relevant SEO keywords for searching for competitor products
        2. Use the search tool to search across multiple platforms using site-specific searches in your query:
           - site:g2.com
           - site:producthunt.com
           - site:reddit.com
           - site:appsumo.com
           - site:alternativeto.net
        3. Only return a clean list of competitor names, nothing else
        Example output: ["Product A", "Product B", "Product C"]""",

        "website_scraper": """You are an expert at finding and extracting key information from company websites.
        For each competitor:
        1. Find their official homepage
        2. Extract and summarize:
           - Main value proposition
           - Key features
           - Pricing plans and details
           - Target audience
        Be thorough but concise in your extraction.""",

        "review_finder": """You are an expert at finding and analyzing product reviews.
        For each competitor:
        1. Search for reviews across multiple sources:
           - Professional review sites
           - Reddit discussions
           - User testimonials
           - G2/Capterra reviews
        2. Summarize the general sentiment
        3. Extract common praise and complaints
        4. Note any standout features or issues mentioned frequently""",

        "competitor_analyzer": """You are an expert market analyst.
        Analyze the compiled competitor data to:
        1. Compare and contrast key features
        2. Identify market positioning
        3. Analyze pricing strategies
        4. Highlight competitive advantages
        5. Note market gaps and opportunities
        
        Always include specific citations when making claims.
        Format your analysis in clear sections with bullet points including and use markdown for readability."""
    }


### Competitor Searcher Agent

In [1]:
import dotenv
dotenv.load_dotenv()

from app.settings import init_aiml, init_openai
init_openai()

In [2]:
from app.agents.stage_2_initial_research.competitor_analysis.competitor_searcher import create_competitor_searcher
competitor_searcher = create_competitor_searcher(chat_history=[])

Using LLM: gpt-4o


In [13]:
handler = competitor_searcher.achat(message="Conduct competitor research on this product idea: Autonomous ai agents that organize social events for lonely people to meet each other")
res = await handler

> Running step 9a3de55f-98cc-49b6-b401-c0ba4df86c9d. Step input: Conduct competitor research on this product idea: Autonomous ai agents that organize social events for lonely people to meet each other
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: competitor_search
Action Input: {'search_query': 'autonomous AI agents for organizing social events for lonely people'}
Observation: {'query': 'autonomous AI agents for organizing social events for lonely people', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.reddit.com/r/AIPsychology/', 'title': 'r/AIPsychology - Reddit', 'content': 'Apr 4, 2023 · This software allows you to run and interact with fully autonomous AI agents right on your Windows PC. 🖥️ W**hat is KinOS? **KinOS is a platform\xa0...Missing:  lonely | Show results with:lonely', 'score': 0.98707944, 'raw_content': None}, {'url': 'https://www.reddit.com/r/OpenAI/commen

/home/kenji/.cache/pypoetry/virtualenvs/app-zBMyfuJ1-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[LOG] Extracted 1 blocks from URL: https://www.producthunt.com/posts/marshmallow-2 block index: 0
[LOG] 🚀 Extraction done for https://www.producthunt.com/posts/marshmallow-2, time taken: 3.50 seconds.
Observation: content=[{'name': 'Marshmallow', 'description': "Marshmallow revolutionizes social by creating intimate, offsite meetups, allowing you to connect with others who share the interests. Using AI, as LLM & GNNs, it analyzes your interactions to ensure every moment counts, making 'any time & where' into experiences.", 'direct_competitor': True, 'source_url': 'https://www.producthunt.com/posts/marshmallow-2', 'relevance_factors': ['Focuses on organizing social events for individuals.', 'Utilizes AI to enhance social interactions and connections.', 'Targets lonely individuals looking to meet others.'], 'interesting_insights': ['The use of AI in social event organization is a growing trend.', "Marshmallow's approach to creating intimate meetups could appeal to a niche market of socia

### Website Scraper Agent

In [1]:
import dotenv
dotenv.load_dotenv()

from app.settings import init_aiml, init_openai
init_openai()

res = """
 {
    "sources": ["https://www.producthunt.com/posts/marshmallow-2"],
    "insights": [
        "The use of AI in social event organization is a growing trend, indicating a potential market for products targeting loneliness and social isolation.",
        "Marshmallow's approach to creating intimate meetups could appeal to a niche market of socially isolated individuals, suggesting that similar solutions could find success in this space."
    ],
    "competitors": [
        {
            "name": "Marshmallow",
            "direct_competitor": true,
            "description": "Marshmallow revolutionizes social by creating intimate, offsite meetups, allowing you to connect with others who share interests. Using AI, including LLM & GNNs, it analyzes interactions to ensure meaningful experiences.",
            "source_url": "https://www.producthunt.com/posts/marshmallow-2",
            "relevance_factors": [
                "Focuses on organizing social events for individuals.",
                "Utilizes AI to enhance social interactions and connections.",
                "Targets lonely individuals looking to meet others."
            ]
        }
    ]
}
"""

In [2]:
from app.agents.stage_2_initial_research.competitor_analysis.competitor_researcher import create_website_scraper
website_scraper = create_website_scraper(chat_history=[])
handler = website_scraper.achat(message="Find and scrape homepage and pricing for the following competitors: " + res)
res = await handler

Using LLM: gpt-4o
> Running step d518c68f-a18e-4fdb-9293-4714f0b18642. Step input: Find and scrape homepage and pricing for the following competitors: 
 {
    "sources": ["https://www.producthunt.com/posts/marshmallow-2"],
    "insights": [
        "The use of AI in social event organization is a growing trend, indicating a potential market for products targeting loneliness and social isolation.",
        "Marshmallow's approach to creating intimate meetups could appeal to a niche market of socially isolated individuals, suggesting that similar solutions could find success in this space."
    ],
    "competitors": [
        {
            "name": "Marshmallow",
            "direct_competitor": true,
            "description": "Marshmallow revolutionizes social by creating intimate, offsite meetups, allowing you to connect with others who share interests. Using AI, including LLM & GNNs, it analyzes interactions to ensure meaningful experiences.",
            "source_url": "https://www.pr

/home/kenji/.cache/pypoetry/virtualenvs/app-zBMyfuJ1-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1792}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Thought: I have successfully gathered the product details from the provided ProductHunt link. Now, I need to search for the pricing information for Marshmallow.
Action: search
Action Input: {'query': 'Marshmallow social meetup platform pricing'}
Observation: {'query': 'Marshmallow social meetup platform pricing', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Marshmallow - bots.babble-ai.com', 'url': 'https://bots.babble-ai.com/marshmallow-2', 'content': "Marshmallow | AI-Powered Event Social Platform Publish a Product Marshmallow AI-Powered Event Social Platform marshmallow.lol Is this product yours? Marshmallow revolutionizes social interactions by creating intimate, offsite meetups that allow you to connect with others who share your interests. Heysummit Run Seamless Events for Your Audience Messaging | Events  Free Whappens - Events Everywhere events, map, online, city, concert, art, sport, meetup Android | Events  Free CelebrateAlly AI Your AI co

/home/kenji/.cache/pypoetry/virtualenvs/app-zBMyfuJ1-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1152}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[LOG] Extracted 1 blocks from URL: https://www.producthunt.com/posts/marshmallow-2 block index: 0
[LOG] 🚀 Extraction done for https://www.producthunt.com/posts/marshmallow-2, time taken: 2.50 seconds.
Observation: content=[{'source': 'Product Hunt', 'rating': 0, 'positive_points': ['Revolutionizes social interactions by creating intimate, offsite meetups.', 'Utilizes AI to analyze interactions, ensuring meaningful connections.', 'Transforms any time and place into valuable experiences.'], 'negative_points': [], 'review_count': 1}] url='https://www.producthunt.com/posts/marshmallow-2' is_error=False error_message=None
> Running step 4484a34a-20cd-48f0-987a-d364369d6d57. Step input: None
Thought: I have gathered the necessary information on Marshmallow, including product details, pricing insights, and reviews. Here is the comprehensive analysis:
Answer: ```json
{
    "product_details": {
        "homepage_url": "https://marshmallow.lol?ref=producthunt",
        "value_proposition": "Mars

In [3]:
res.response

'```json\n{\n    "product_details": {\n        "homepage_url": "https://marshmallow.lol?ref=producthunt",\n        "value_proposition": "Marshmallow revolutionizes social by creating intimate, offsite meetups, allowing you to connect with others who share interests. Using AI, including LLM & GNNs, it analyzes interactions to ensure meaningful experiences.",\n        "key_features": [\n            "AI-powered event social platform",\n            "Intimate offsite meetups",\n            "Connect with others who share interests",\n            "Analyzes interactions using LLM & GNNs"\n        ],\n        "target_audience": [\n            "Individuals looking to connect socially",\n            "Event organizers",\n            "Communities with shared interests"\n        ],\n        "tech_stack": [\n            "AI",\n            "Large Language Models (LLM)",\n            "Graph Neural Networks (GNNs)"\n        ]\n    },\n    "pricing_info": {\n        "pricing_model": "unknown",\n        "

In [12]:
parsed_response = CompetitorSearchResponse.model_validate_json(res.response)
parsed_response



CompetitorSearchResponse(sources=[], insights=['The concept of autonomous robots cleaning autonomous cars appears to be a novel idea with limited existing competition. Current search results do not show specific products or companies addressing this niche directly.', 'There might be adjacent technologies or solutions in development, such as autonomous car washes or robotic cleaning devices for other purposes, but they have not been explicitly linked to the cleaning of autonomous vehicles.', 'The lack of direct competitors indicates a potential opportunity for innovation and development in this area, suggesting that the market is not yet saturated.'], competitors=[])

### Competitor Analysis Workflow

In [1]:
from app.workflows.single import AgentRunEvent
import dotenv
dotenv.load_dotenv()

from app.settings import init_aiml, init_openai
init_openai()

from app.agents.stage_2_initial_research.competitor_analysis.workflow import create_competitor_analysis

workflow = create_competitor_analysis(chat_history=[])
handler = workflow.run(input="Perform a competitive analysis on the following product idea: Autonomous ai agents that organize social events for lonely people to meet each other")
async for event in handler.stream_events():
    if type(event) is AgentRunEvent:
        print(event.msg)
        
res = await handler

Using LLM: gpt-4o
Using LLM: gpt-4o
Using LLM: gpt-4o
Using LLM: gpt-4o


/home/kenji/.cache/pypoetry/virtualenvs/app-zBMyfuJ1-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "<lambda>" shadows an attribute in parent "BaseModel"
  warnings.warn(


Using LLM: gpt-4o
> Running step 137d58fb-c477-4cf5-8979-a0394f2912e6. Step input: Conduct competitor research on this product idea: Perform a competitive analysis on the following product idea: Autonomous ai agents that organize social events for lonely people to meet each other
Thought: The current language of the user is English. I need to use a tool to help me find competitors for the product idea of "Autonomous AI agents that organize social events for lonely people to meet each other."
Action: competitor_search
Action Input: {'search_query': 'autonomous AI agents for social event organization'}
Observation: {'query': 'autonomous AI agents for social event organization', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.reddit.com/r/ArtificialInteligence/comments/1dk5wh2/ai_agents_are_all_you_need/', 'title': 'AI Agents Are All You Need : r/ArtificialInteligence - Reddit', 'content': "Jun 20, 2024 · In an LLM-powered autonomous agent syste

/home/kenji/.cache/pypoetry/virtualenvs/app-zBMyfuJ1-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Thought: I have extracted the product details successfully. Now, I need to search for the pricing information of Fellow.app and then extract it.
Action: search
Action Input: {'query': 'Fellow.app pricing'}
Observation: {'query': 'The Loneliness Network platform company official site', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'The complexity of loneliness - PMC - PubMed Central (PMC)', 'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC6179015/', 'content': 'An official website of the United States government ... and social loneliness as the lack of a social network, the absence of a circle of people that allows an individual to develop a sense of belonging, of company, of being part of a community. Both in daily life and in the research area, various researchers have referred to "loneliness', 'score': 0.9859364, 'raw_content': None}, {'title': 'How Panion is tackling loneliness with connection', 'url': 'https://www.lonelinessawarenessweek.org/post/

In [ ]:
res